# LinkedIn Scraper

Link to code: https://levelup.gitconnected.com/linkedin-scrapper-a3e6790099b5

In [2]:
# Dependencies
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import requests
from bs4 import BeautifulSoup
import re
import time
import json

In [3]:
# Set driver path
path = '/Users/carlos/Documents/My Working Directory/chromedriver' #ChromeDriver path
# Initialize chromedriver to driver variable
browser = webdriver.Chrome(path)

In [4]:
#Function to log into LinkedIn
def log_In():
    #Open LinkedIn login page
    browser.get('https://www.linkedin.com/login?fromSignIn=true&trk=guest_homepage-basic_nav-header-signin')

    #Ask user to enter login info
    User_username = input('Please enter your LinkedIn username:\n')
    User_password = input('Please enter your LinkedIn password:\n')

    #Enter login info:
    elementID = browser.find_element_by_id('username')
    elementID.send_keys(User_username)

    elementID = browser.find_element_by_id('password')
    elementID.send_keys(User_password)

    elementID.submit()#Clicks login button

In [5]:
# ckelaid@g.clemson.edu
# ck6977418029
log_In()

Please enter your LinkedIn username:
ckelaid@g.clemson.edu
Please enter your LinkedIn password:
ck6977418029


In [10]:
#Function to search for a job title, and scrape the job
#information of the first page of job listings 
job_titles = []
def Search_for_job():

    #Ask user to input job type he wishes to search for (data science internships)
    jobID = input('What job do you want to search for?\n') 
    
    #Split jobID in list of strings
    which_jobID = jobID.split()
    print(which_jobID)
    # Want add '%20' in-place of whitespace to search for see_all_page link
    space = '%20'
    see_all_jobID = ''
    for i in range(len(which_jobID)):
        see_all_jobID = see_all_jobID+which_jobID[i]+space

    print(see_all_jobID)
    see_all_jobID = see_all_jobID[:-3] #Removes the %20 at the
    print(see_all_jobID)

    # see_all *Job-Title-Page*
    see_all_page = browser.get('https://www.linkedin.com/jobs/search/?keywords='+see_all_jobID)
    
    # Store in r the see_all_page link to use bs to scrape
    r = requests.get('https://www.linkedin.com/jobs/search/?keywords='+see_all_jobID)
    #Parse html using bs
    soup = BeautifulSoup(r.text, 'html.parser')

    # Using regex method, we grab all the <a> & class="base-card__full-link"
    # text in Job-title page
    regex = re.compile(r'[/]([a-z]|[A-Z])\w+')
    results = soup.find_all('a', {'class':"base-card__full-link"}, href = regex)
    
    # Iterating through our results array to grab desired links and store them
    # in job_titles
    #job_titles = []
    for i in range(len(results)-1):
        job_titles.append(results[i].attrs['href'])
    
    #return job_titles

In [11]:
# data science internships 2022
# data science 2022
Search_for_job()

What job do you want to search for?
data science 2022
['data', 'science', '2022']
data%20science%202022%20
data%20science%202022


In [9]:
len(job_titles)

21

In [36]:
job_titles

['https://www.linkedin.com/jobs/view/assistant-and-assistant-associate-professor-data-science-and-analytics-at-kennesaw-state-university-school-of-data-science-and-analytics-2838545212?refId=FL1WEsZ29nx4TYJ9fV87qg%3D%3D&trackingId=rEK1JwN2ESJh7OhGeijvNw%3D%3D&position=1&pageNum=0&trk=public_jobs_jserp-result_search-card',
 'https://www.linkedin.com/jobs/view/data-scientist-machine-learning-intern-prototypes-at-primer-ai-2872119547?refId=FL1WEsZ29nx4TYJ9fV87qg%3D%3D&trackingId=MH75O3x8j6PrvH8PFpy9kg%3D%3D&position=2&pageNum=0&trk=public_jobs_jserp-result_search-card',
 'https://www.linkedin.com/jobs/view/natural-resource-data-scientist-at-ecotrust-2874382581?refId=FL1WEsZ29nx4TYJ9fV87qg%3D%3D&trackingId=Ntx0OhKbHlEcOAXg9Gc0sA%3D%3D&position=3&pageNum=0&trk=public_jobs_jserp-result_search-card',
 'https://www.linkedin.com/jobs/view/data-science-intern-summer-2022-at-reddit-inc-2818883615?refId=FL1WEsZ29nx4TYJ9fV87qg%3D%3D&trackingId=97qNxoddEeLa28Hd1h0xaA%3D%3D&position=4&pageNum=0&trk=p

In [10]:
# The function that takes our job-title links as an input
# The input (aa) is the job_titles array
Date_Posted = []
Job_Info = []
    
def get_job_info(aa):    
    for a in aa:
        title_info = browser.get(a)#Sends me on job_titles[0]'s job-title individual page
        # Parse info on individual job_title page
        src = requests.get(a)
        soup2 = BeautifulSoup(src.text, 'html.parser')
        # Parse json code block
        data = soup2.find_all('script', {'type':'application/ld+json'})[0].text
        info = json.loads(data)
        # Add date job was posted to list
        Date_Posted.append(info['datePosted'][0:10])
        # Parse job_description to get rid of embedded tags
        soup3 = BeautifulSoup(info['description'])
        soup4 = BeautifulSoup(soup3.text, 'html.parser')
        # Replace <br> tags by white space
        for e in soup4.find_all('br'):
            e.replace_with(' ')
        
        #Get only text back without imbedded tags & store it in Job_Info list/array
        Job_Info.append(soup4.text)

In [11]:
# Calling the function
get_job_info(job_titles)

IndexError: list index out of range

In [12]:
len(Job_Info)

18

### Store scraped information into pandas data frame

In [13]:
# Dependencies
import numpy as np
import pandas as pd

In [14]:
# Store all extracted job info in Pandas dataframe
df = pd.DataFrame(np.array(Job_Info), columns=['Job Information'])
# Let's add the Date_Posted as well as a column in the dataframe
df['Job Posted on'] = Date_Posted
df.head()

,Job Information,Job Posted on
0,Kennesaw State University is now accepting app...,2021-12-16
1,The world is changing at an accelerating rate....,2022-01-08
2,Position Title: Natural Resource Data Scientis...,2022-01-11
3,"Reddit is a network of more than 100,000 commu...",2021-12-25
4,About the jobJob Description: We’re looking fo...,2022-01-11


In [15]:
df.tail()

,Job Information,Job Posted on
13,"From the start, we’ve made it our mission to b...",2022-01-08
14,Company Description Blend360 is a world class...,2022-01-07
15,We are looking for Data Scientist Interns to #...,2022-01-11
16,"Monicat Data is a women-owned, women lead comp...",2022-01-10
17,Job Title: Data Analyst Department/Group: Sys...,2022-01-03
